## Load common code

In [ ]:
"""
#For testing, leave commented
swivel = 1
swivel_order = [2, 1, 22, 7, 16, 6, 3, 12, 10, 9, 4, 5, 15, 8, 14, 0, 20, 13, 19, 11, 18, 21, 17]

desired_file_prefix = "NF_GENDER" 

try:
   if (len(desired_file_output) == 0):
       desired_file_output = ""
except:
    desired_file_output = "" 
    
by_year = 0
year    = 1    # 0->7

normalize_data=True

lnconvert=False

ocup_sel = 1
       
desired_file_output = "./NF_SUB_GENDER/NF_GENDER_" + str(2011+year) + "_"    
"""

In [ ]:
#For testing
#desired_file_prefix = "NF_GENDER" 
#normalize_data=True
#desired_file_output = ""

%run ./common-code.ipynb

## Select Generic Database to work with

In [ ]:
genericDB_files_dict = {}

#desired_file_prefix = "F_GENDER" 

## Create above database dictionary

In [ ]:
import os

filelist = os.listdir('../DATA')

for file in filelist:
    if (file.find(desired_file_prefix)==0):
        print(file)
        fname = "../DATA/"+file
        afile = open(fname, "r")
        data = afile.read()
        name = file.split("_all")[0]
        genericDB_files_dict.setdefault(name, []).append(data)
        #print(data)
        afile.close()

In [ ]:
#print(genericDB_files_dict)
print(genericDB_files_dict.keys())

# Create a dictionary of soc_codes
#
genericDB_dict = {}

for db in genericDB_files_dict.keys():
    
    genericDB = genericDB_files_dict[db]
    
    #print(genericDB[0])    
    line = genericDB[0].replace("\"","")
    genericDB_array= line.split("\n")
    
    #print(line)
    #print("*********")
    #print(genericDB_array)

# Analysis

In [ ]:
# Create a dictionary of soc_codes
#
genericDB_dict = {}


# Create Dictionary of SOC codes
#
for db in genericDB_files_dict.keys():
    
    genericDB = genericDB_files_dict[db]
    
    #print(genericDB[0])    
    line = genericDB[0].replace("\"","")
    genericDB_array= line.split("\n")
    #print(genericDB_array[12:])
    
    # Line 12 is where the header stops and the data starts
    #
    for line in genericDB_array[12:]:
        line_array = line.split(",")
        #key  = line_array[0].    [0:7].replace("-","")
        key  = line_array[0][6:12]
        data = line_array[1:]
        #print(line_array)
        #print(key,data)

        # Replace empty values with zero
        if (len(key) > 1):
            #print(key,data[0:8])
            for index in range(0,8):
                if (len(data[index]) == 0):
                    data[index] = '0'
            #print(key,data[0:8])
        
        # Convert to Integers
        if (len(key) > 1):
            #print(key,data[0:8])
            for index in range(0,8):
                data[index] = int(data[index])
            #print(key,data[0:8])   
            
        # Create dictionary entry
        if (len(key) > 1):
            genericDB_dict.setdefault(db+"_"+key, []).append(data[0:8])
    

## Scatter plot level 1 RELATIVE average count of DATABASE data for soc groups

In [ ]:
%matplotlib inline

In [ ]:
# Create list of new level 1 codes

soc_new_lv1 = []

for key in soc_new_code:
    
    # Grab first two chars of code
    lvindex = key[0][0:2]
    
    #Eliminate "00" ALL
    #if (lvindex.find("00") < 0):
    
    if lvindex not in soc_new_lv1:
        soc_new_lv1.append(lvindex)
        #print(lvindex)

#print(soc_new_lv1)

In [ ]:
#print("HERE")
#print(print(genericDB_files_dict.keys()))

In [2]:
lastfound = ""
pcov      = []


no_genericDB_data_dict = {}

#print("HERE")
#print(genericDB_dict.keys())

# running generic_totals for 2011 to 2018 
for db in genericDB_files_dict.keys():
    x = []
    y = []
    count = 0 
    zero_count = 0
    
    #print(db)
        
    for ocup in range(ocup_sel,ocup_sel+1):
        
        index = soc_new_lv1[ocup]
        
        
        print("############ OCCUPATION ############")
        code_found = "NO_CODE_FOUND"
        for code in soc_new_code:
            #print(code[0][0:2])
            if (code[0][0:2].find(index) >= 0):
                code_found = str(code[0]) + " " + str(code[1])
                #print(code_found)
        #print(" ")

    
        if (index.find("00") == 0):
            if (desired_file_prefix[0:3] == "NF_"):
                new_key = db+"_"+"0XXXXX"
            else:
                new_key = db+"_"+index  # +"0000"
        else:
            if (desired_file_prefix[0:3] == "NF_"):
                new_key = db+"_"+index  # +"0000"
            else:
                new_key = db+"_"+index  # +"XXXX"
            
        #print(new_key)
    
        if (len(index) > 0):
            generic_totals = []
            
            for db_key in genericDB_dict.keys():
                #print(db_key,new_key)
                if (db_key.find(new_key) == 0) and (db_key[-1].find("X") < 0) and (db_key[-4:].find("0000") < 0):
                    #print(db_key[-4:])
                    #print(db_key,new_key, db_key.split(new_key)[1][0])
                    print("found "+db_key)
                    #print(genericDB_dict[db_key][0])
                    #generic_totals = generic_totals + np.array(genericDB_dict[db_key][0])
                      
                    #print(year, genericDB_dict[db_key][0])
                    
                    gen_value = genericDB_dict[db_key][0][year]
                    if (gen_value > 0.0):
                        generic_totals.append(gen_value)
                        #print("*gt:",generic_totals)  
                        
                        
                    soc3 = str(db_key.split(new_key)[1][0:2])
                    if (soc3.find(lastfound) < 0):
                        #print("lastfound, db_key",lastfound,db_key)
                        pcov_avg = np.array(pcov).std() / np.array(pcov).mean()
                        print("SOC3 change detected: pcov_avg:"+str(pcov_avg))
                        pcov = []
                        pcov.append(generic_totals)  
                        lastfound = str(db_key.split(new_key)[1][0:2])
                    else:
                        pcov.append(generic_totals)  
                        lastfound = str(db_key.split(new_key)[1][0:2])
                        
            pcov_avg = np.array(pcov).std() / np.array(pcov).mean()
            print("SOC3 change detected: pcov_avg:"+str(pcov_avg))
            pcov = []
                    
            print("generic_totals:"+str(generic_totals)) 
                        
            print("#################DB:",db)
             
            tmp_lst = generic_totals
               
            if (db.find("_GENDER")>=0 and db.find("_F")>=0):
                ocup_total = total_female[ocup-1]*1000
            
            elif (db.find("_GENDER")>=0 and db.find("_M")>=0):
                ocup_total = total_male[ocup-1]*1000
            
            elif (db.find("_R")>=0 and db.find("_White")>=0):
                ocup_total = total_white[ocup-1]*1000
            
            elif (db.find("_R")>=0 and db.find("_Black")>=0):
                ocup_total = total_black[ocup-1]*1000
            
            elif (db.find("_R")>=0 and db.find("_Asian")>=0):
                ocup_total = total_asian[ocup-1]*1000
            
            elif (db.find("_R")>=0 and db.find("_Hispanic")>=0):
                ocup_total = total_asian[ocup-1]*1000
            
            else:
                print("########## NEED TO EVAL ##################")
                print("########## NEED TO EVAL ##################")
                ocup_total = np.sum(tmp_lst[1:])  # total_total*1000
                if (ocup_total==0):
                    ocup_total = 1
                    
            print("ocup_total, tmp_lst", ocup_total, tmp_lst)    
            
            #ocup_total = 1
            
            tmp_lst = np.array(tmp_lst) / ocup_total # total_total*1000
            #tmp_lst = tmp_lst / ocup_total
            
            print("tmp_lst/ocup_total", tmp_lst)    
            
            #cov.append(tmp_lst.std() / tmp_lst.mean()) 
            cov = cov + (tmp_lst.tolist())
            print("cov_array:",cov)
            
            #tmp_lst = np.array(generic_totals).sum()
            tmp_lst = tmp_lst.mean()
            #tmp_lst = tmp_lst.var()
            
            print("tmp_lst.mean()", tmp_lst)     
                                              
            #print(generic_totals)   
            #relative_totals = fatal_totals / generic_totals
            #relative_totals = generic_totals
            value = tmp_lst
            
            count=count+1
            
            print("*",value)

            x.append(count-1)
            
            #print("***", y, value)
            #y = y + list(value)      # Merge lists
            y.append(value)
            
            if (np.mean(value) <= no_data_threshold):
                zero_count = zero_count + 1

               
    print("**",y)
    
    # Eliminate the first value
    #y = y[1:]
    #x = x[1:]            
                
    #Append data set
    label = str(year+2011)+"_"+str(code_found)+"_"+str(db)
    global_y_soc1.append([label,y])
    
    print("**********",global_y_soc1)
            
    # Check for empty sets
    if (zero_count > 10):
        if (db not in global_soc1_empty):
            global_soc1_empty.append(db) 
         
    #print("****",db,y)
    #print(" ")

    

NameError: name 'genericDB_files_dict' is not defined

## Functions

In [ ]:
#def color_corr_green(val):
#    """
#    Takes a scalar and returns a string with
#    the css property `'color: red'` for negative
#    strings, black otherwise.
#    """
#    color = 'green' if val > 0.8 else 'black'
#   return 'color: %s' % color